In [4]:
import sys
sys.path.append('../implementation/')
import ast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.special as sp
from tqdm import tqdm
import time
from weighted_k_nearest_neighbors import WeightedKNN
from util import flatten_list
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Loading the underlying data and user interaction data
data_path = '../data/vast_2011_challenge/vast_data_sample_reduced.pkl'
bookmarks_ui_data_path = '../data/vast_2011_challenge/bookmark_interactions_clean.pkl'
hovers_ui_data_path = '../data/vast_2011_challenge/hover_interactions.pkl'
hovers = False
if hovers:
    output_file_path = '../output/vast/vast_11_hovers_knn.pkl'
    interaction_data = pd.read_pickle(hovers_ui_data_path)
else:
    output_file_path = '../output/vast/vast_11_knn.pkl'
    interaction_data = pd.read_pickle(bookmarks_ui_data_path)
    interaction_data = interaction_data[interaction_data['experimental_group'] == 'control']
    interaction_data = interaction_data.reset_index(drop=True)

underlying_data = pd.read_pickle(data_path)

ks = [1, 5, 10, 20, 50, 100]
d_attributes = ['topic']
c_attributes = [['latitude', 'longitude']]

In [6]:
knn_results = pd.DataFrame()

for participant_index, row in interaction_data.iterrows():
    print(f'Processing user {row.user} task {row.experimental_group}')
    results = {'participant_id': row.user, 'task': row.experimental_group}
    knn_model = WeightedKNN(underlying_data, 
                            [['latitude', 'longitude']], 
                            ['topic'])
    predicted = pd.DataFrame()
    rank_predicted = []
    for i in tqdm(range(len(interaction_data.iloc[participant_index].interaction_session))):
        interaction = interaction_data.iloc[participant_index].interaction_session[i]
        knn_model.update(interaction)

        if i < len(interaction_data.iloc[participant_index].interaction_session) - 1:
            probability_of_next_point = knn_model.predict()
            next_point = interaction_data.iloc[participant_index].interaction_session[i+1]
            predicted_next_dict = {}
            for k in ks:
                predicted_next_dict[k] = (next_point in probability_of_next_point.nlargest(k).index.values)
            predicted = predicted.append(predicted_next_dict, ignore_index=True)
            sorted_prob = probability_of_next_point.sort_values(ascending=False)
            rank, = np.where(sorted_prob.index.values == next_point)
            rank_predicted.append(rank[0] + 1)
            
    ncp = predicted.sum()/len(predicted)
    results['rank'] = rank_predicted 
    
    for col in ncp.index:
        results[f'ncp-{col}'] = ncp[col]
        
    bias = knn_model.get_attribute_bias()
    for col in bias.columns:
        results[f'bias-{col}'] = bias[col].to_numpy()
        
    knn_results = knn_results.append(results, ignore_index=True)
    
knn_results.to_pickle(output_file_path)

Processing user ottNn7b8Wt1EL-I0bfFbOA task control
Computing neighborhood matrix for latitude___longitude
Computing neighborhood matrix for topic


100%|██████████| 45/45 [00:01<00:00, 26.05it/s]


Processing user Tn8m4yLs6E0GDO6sggwYTw task control
Computing neighborhood matrix for latitude___longitude
Computing neighborhood matrix for topic


100%|██████████| 147/147 [00:05<00:00, 25.29it/s]


Processing user SW8GwZ1tt4Z5DiK3J4rw2g task control
Computing neighborhood matrix for latitude___longitude
Computing neighborhood matrix for topic


...

100%|██████████| 59/59 [00:01<00:00, 42.49it/s]


Processing user RZKgmh54IrTo2bVMpg0L9w task control
Computing neighborhood matrix for latitude___longitude
Computing neighborhood matrix for topic


100%|██████████| 38/38 [00:00<00:00, 44.64it/s]


Processing user Z2yONYSDzxqhG_m_AZAnog task control
Computing neighborhood matrix for latitude___longitude
Computing neighborhood matrix for topic


100%|██████████| 117/117 [00:02<00:00, 40.11it/s]
